# Klasifikasi Berita

## Crawling Portal Berita Antaranews

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import random

- request: Library ini digunakan untuk mengirim HTTP requests (seperti GET, POST, PUT, DELETE) dan menerima respons dari server. Dengan menggunakan Requests, Anda dapat mengakses dan mengambil data dari URL, mengirim data form, dan berinteraksi dengan API.

- beautifulSoup: Beautiful Soup adalah library Python yang digunakan untuk mengambil data dari HTML dan XML. Ini menyediakan cara yang mudah untuk menjelajahi, mencari, dan memanipulasi struktur data dokumen web. Beautiful Soup bekerja dengan parser HTML, membantu Anda mengekstrak informasi dari halaman web dengan lebih mudah.

- csv: Modul CSV di Python menyediakan fungsi untuk membaca dan menulis file CSV. Ini memungkinkan Anda untuk dengan mudah bekerja dengan data dalam format CSV, yang umumnya digunakan untuk menyimpan data tabel, terutama dalam konteks analisis data dan pertukaran data antar program.

- random: Library ini menyediakan fungsi-fungsi untuk menghasilkan angka acak. Dalam konteks web scraping atau pengujian, Anda mungkin menggunakan library ini untuk memilih elemen acak dari daftar atau untuk menyimulasikan perilaku yang tidak terduga.



In [ ]:
# masukkan url
response = requests.get("https://www.antaranews.com/")
# Isi teks dari respons HTTP yang diterima dari server web setelah melakukan permintaan GET.
soup = BeautifulSoup(response.text, 'html.parser')
# menemukan semua list yang berisi link kategori
first_page = soup.findAll('li',"dropdown mega-full menu-color1")

# menyimpan kategori
save_categori = []
for links in first_page:
  categori = links.find('a').get('href')
  save_categori.append(categori)
# save_categori

# categori yang akan dicari terdapat pada indeks 1,3,6
categori_search = [save_categori[3],save_categori[6],save_categori[9]]
categori_search

['https://www.antaranews.com/ekonomi',
 'https://www.antaranews.com/olahraga',
 'https://www.antaranews.com/hiburan']

In [ ]:
# Inisialisasi list untuk menyimpan data berita
datas = []

# Iterasi melalui halaman berita
for ipages in range(1, 3):

    # Iterasi melalui setiap kategori berita
    for beritas in categori_search:
        # Permintaan untuk halaman berita
        response_berita = requests.get(beritas + "/" + str(ipages))
        namecategori = beritas.split("/")

        # Parsing halaman berita dengan BeautifulSoup
        soup_berita = BeautifulSoup(response_berita.text, 'html.parser')
        pages_berita = soup_berita.findAll('article', {'class': 'simple-post simple-big clearfix'})

        # Iterasi melalui setiap artikel dalam halaman berita
        for items in pages_berita:
            # Mendapatkan link artikel
            get_link_in = items.find("a").get("href")

            # Request untuk halaman artikel
            response_artikel = requests.get(get_link_in)
            soup_artikel = BeautifulSoup(response_artikel.text, 'html.parser')

            # Ekstraksi informasi dari halaman artikel
            judul = soup_artikel.find("h1", "post-title").text if soup_artikel.findAll("h1", "post-title") else ""
            label = namecategori[-1]
            date = soup_artikel.find("span", "article-date").text if soup_artikel.find("span", "article-date") else "Data tanggal tidak ditemukan"

            trash1 = ""
            cek_baca_juga = soup_artikel.findAll("span", "baca-juga")
            if cek_baca_juga:
                for bacas in cek_baca_juga:
                    text_trash = bacas.text
                    trash1 += text_trash + ' '

            artikels = soup_artikel.find_all('div', {'class': 'post-content clearfix'})
            artikel_content = artikels[0].text if artikels else ""
            artikel = artikel_content.replace("\n", " ").replace("\t", " ").replace("\r", " ").replace(trash1, "").replace("\xa0", "")

            author = soup_artikel.find("p", "text-muted small mt10").text.replace("\t\t", "") if soup_artikel.findAll("p", "text-muted small mt10") else ""

            # Menambahkan data artikel ke dalam list
            datas.append({'Tanggal': date, 'Penulis': author, 'Judul': judul, 'Artikel': artikel, 'Label': label})


In [ ]:
# Iterasi melalui halaman berita
for ipages in range(1, 3):
    # Mengecek apakah iterasi saat ini kurang dari 2
    if ipages < 2:
        # Menampilkan jumlah data yang telah berhasil di-crawling
        print(f'Data berhasil dicrawling sebanyak : {len(datas)}')

In [ ]:
# Menyimpan data dalam bentuk CSV
csv_filename = 'berita_politik_antaranews.csv'
# Membuka file CSV untuk ditulis ('w' mode) dengan encoding utf-8
with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    # Menentukan kolom (fieldnames) untuk file CSV
    fieldnames = ['Tanggal', 'Penulis', 'Judul', 'Artikel', 'Label']
    # Membuat objek DictWriter untuk menulis data ke file CSV
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    # Menulis header (baris pertama) ke file CSV
    csv_writer.writeheader()
    # Iterasi melalui data yang akan disimpan (diasumsikan datas sudah didefinisikan sebelumnya)
    for entry in datas:
        # Menulis setiap baris data ke file CSV
        csv_writer.writerow(entry)
# Menampilkan pesan bahwa data telah disimpan dalam file CSV
print(f"Data telah disimpan dalam file {csv_filename}")

## Import library/package yang dibutuhkan (Memproses Data)

In [ ]:
import pandas as pd
import re

- Pandas adalah library open-source yang ditulis dalam bahasa pemrograman Python untuk analisis data dan manipulasi data. Dikembangkan oleh Wes McKinney, Pandas menyediakan struktur data yang efisien dan mudah digunakan untuk memanipulasi dan menganalisis data numerik dan tabular. Pandas sangat populer di kalangan data scientist, analis data, dan pengembang perangkat lunak karena kemudahan penggunaannya dan dukungan yang luas dalam ekosistem Python

- re adalah library Python yang menyediakan dukungan untuk ekspresi reguler (regular expressions). Ekspresi reguler adalah urutan karakter yang membentuk pola pencarian. Modul re memungkinkan pencocokkan pola dengan string dan melakukan berbagai operasi seperti pencarian, pencocokan, dan manipulasi string berdasarkan pola tertentu.

## Load data

Memanggil dataset yang akan digunakan dan ditampilkan dalam bentuk dataframe

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/dennywr/cobaprosaindata/main/berita_antaranews_fhd.csv')

In [ ]:
df = pd.DataFrame(data)
df.head(5)

,Tanggal,Penulis,Judul,Artikel,Label
0,"Selasa, 5 Desember 2023 08:09 WIB",Pewarta: Benardy FerdiansyahEditor: Ahmad Wija...,PGE sebut panas bumi merupakan energi terbaruk...,Ini dikarenakan panas bumi memiliki ketersedi...,ekonomi
1,"Selasa, 5 Desember 2023 08:07 WIB",Penerjemah: Citro AtmokoEditor: Ahmad WijayaCO...,Dolar menguat didorong kenaikan imbal hasil ob...,Dengan perkiraan pelonggaran sebesar 135 basi...,ekonomi
2,"Selasa, 5 Desember 2023 07:00 WIB",NaN,Sektor pariwisata bersiap sambut libur Natal 2023,Kementerian Pariwisata dan Ekonomi Kreatif...,ekonomi
3,"Selasa, 5 Desember 2023 06:51 WIB",Pewarta: SumarwotoEditor: Ahmad WijayaCOPYRIGH...,KAI: Jalur hilir Purwokerto-Cirebon sudah bisa...,Jalur hilir di titik yang terdampak longsor s...,ekonomi
4,"Selasa, 5 Desember 2023 06:48 WIB",Pewarta: SubagyoEditor: Ahmad WijayaCOPYRIGHT ...,Produsen AMDK komitmen atasi krisis air bersih...,Permasalahan air bersih sudah selayaknya menj...,ekonomi


In [ ]:
df['Artikel'] = df['Artikel'].fillna('').astype(str)

## Preprocessing

Preprocessing merupakan serangkaian langkah atau tahapan yang dilakukan untuk membersihkan, merapihkan, dan mengubah data teks mentah menjadi bentuk yang lebih sesuai untuk analisis atau pemodelan. Proses ini penting karena data teks seringkali datang dalam bentuk yang tidak terstruktur dan memerlukan persiapan khusus sebelum dapat digunakan secara efektif

- Hapus karakter spesial pada teks

> Karakter spesial adalah karakter yang memiliki makna khusus seperti : / $ |


In [ ]:
def removeSpecialText (text):
  text = text.replace('\\t',"").replace('\\n',"").replace('\\u',"").replace('\\',"").replace('None',"")
  text = text.encode('ascii', 'replace').decode('ascii')
  return text.replace("http://"," ").replace("https://", " ")
df['Artikel'] = df['Artikel'].apply(removeSpecialText)
print(df['Artikel'])

0        Ini dikarenakan panas bumi memiliki ketersedi...
1        Dengan perkiraan pelonggaran sebesar 135 basi...
2           Kementerian Pariwisata dan Ekonomi Kreatif...
3        Jalur hilir di titik yang terdampak longsor s...
4        Permasalahan air bersih sudah selayaknya menj...
                              ...                        
1075         Jakarta (ANTARA) - Grup idola perempuan N...
1076         Jakarta (ANTARA) - Film horor ?Pemukiman ...
1077         Jakarta (ANTARA) - Aktor "Grey's Anatomy"...
1078                                                     
1079                         ANTARA - Pencinta komik J...
Name: Artikel, Length: 1080, dtype: object


- Hapus tanda baca pada teks

> Tanda baca adalah simbol atau karakter yang digunakan dalam penulisan untuk memberikan arti atau menandai struktur kalimat, misalnya . , ? !

In [ ]:
def removePunctuation(text):
  text = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text)
  return text

df['Artikel'] = df['Artikel'].apply(removePunctuation)
df['Artikel'].head(20)

0      Ini dikarenakan panas bumi memiliki ketersedi...
1      Dengan perkiraan pelonggaran sebesar 135 basi...
2         Kementerian Pariwisata dan Ekonomi Kreatif...
3      Jalur hilir di titik yang terdampak longsor s...
4      Permasalahan air bersih sudah selayaknya menj...
5          Jakarta  ANTARA    Sejumlah kabar mulai d...
6      Seluruh penyelenggara tersebut terbagi dalam ...
7                          ANTARA   Alokasi Anggaran...
8          Medan  ANTARA    Perum Bulog Kanwil Sumat...
9                                                      
10         Jakarta  ANTARA    Menteri Badan Usaha Mi...
11         Kota Bandung  ANTARA    Menteri Koperasi ...
12     Optimalisasi layanan tentunya untuk mengantis...
13         Jakarta  ANTARA    PT PLN  Persero  melal...
14     Jadi total targetnya Natal dan Tahun Baru ini...
15         Jakarta  ANTARA    Pebulu tangkis tunggal...
16         Jakarta  ANTARA    Wakil Indonesia dalam ...
17         Banyuwangi  ANTARA    Seorang atlet k

- Hapus angka pada teks

> Angka adalah simbol matematika yang digunakan untuk menyatakan nilai kuantitatif atau jumlah, misalnya 1, 2, 3, 4, 5, 6

In [ ]:
def removeNumbers (text):
  return re.sub(r"\d+", "", text)
df['Artikel'] = df['Artikel'].apply(removeNumbers)
df['Artikel']

0        Ini dikarenakan panas bumi memiliki ketersedi...
1        Dengan perkiraan pelonggaran sebesar  basis p...
2           Kementerian Pariwisata dan Ekonomi Kreatif...
3        Jalur hilir di titik yang terdampak longsor s...
4        Permasalahan air bersih sudah selayaknya menj...
                              ...                        
1075         Jakarta  ANTARA    Grup idola perempuan N...
1076         Jakarta  ANTARA    Film horor  Pemukiman ...
1077         Jakarta  ANTARA    Aktor  Grey s Anatomy ...
1078                                                     
1079                         ANTARA   Pencinta komik J...
Name: Artikel, Length: 1080, dtype: object

- Ubah semua huruf pada teks menjadi huruf kecil

> Untuk menghindari ambiguitas dan mempermudah pemrosesan teks, terutama dalam konteks perbandingan string dan pencarian teks. Dalam banyak kasus, case folding digunakan untuk mengurangi kompleksitas dan membuat perbandingan teks menjadi lebih konsisten (keseragaman)

In [ ]:
def casefolding(Comment):
  Comment = Comment.lower()
  return Comment
df['Artikel'] = df['Artikel'].apply(casefolding)
df['Artikel']

0        ini dikarenakan panas bumi memiliki ketersedi...
1        dengan perkiraan pelonggaran sebesar  basis p...
2           kementerian pariwisata dan ekonomi kreatif...
3        jalur hilir di titik yang terdampak longsor s...
4        permasalahan air bersih sudah selayaknya menj...
                              ...                        
1075         jakarta  antara    grup idola perempuan n...
1076         jakarta  antara    film horor  pemukiman ...
1077         jakarta  antara    aktor  grey s anatomy ...
1078                                                     
1079                         antara   pencinta komik j...
Name: Artikel, Length: 1080, dtype: object

- NLTK (Natural Language Toolkit): salah satu pustaka Python yang sangat populer untuk pemrosesan bahasa alami (Natural Language Processing). NLTK menyediakan berbagai alat dan sumber daya untuk bekerja dengan teks, seperti tokenisasi, stemming, lemmatization, analisis sentimen, dan lain sebagainya

- nltk.download('punkt'): perintah untuk mengunduh sumber daya punkt dari NLTK. Punkt merupakan model tokenisasi dari NLTK yang digunakan untuk memisahkan teks menjadi token (kata atau bagian teks lainnya). Tokenisasi adalah proses memecah teks menjadi unit-unit yang lebih kecil, seperti kata atau kalimat.

- nltk.corpus.stopwords: NLTK menyediakan daftar kata-kata umum yang seringkali tidak membawa banyak informasi atau disebut sebagai "stop words". Stop words adalah kata-kata umum seperti "the", "and", "is", dll., yang sering kali dihapus dalam tahapan pra-pemrosesan teks karena jarang memberikan nilai informasi dalam analisis.

- nltk.download('stopwords'): perintah untuk mengunduh daftar kata-kata stop words dari NLTK. Setelah diunduh, Anda dapat mengaksesnya menggunakan nltk.corpus.stopwords.words('language'), dengan 'language' diganti sesuai bahasa yang Anda inginkan (contoh: 'indonesia' untuk bahasa Indonesia).

In [ ]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

- Penghapusan Stopwords

> Untuk meningkatkan kualitas representasi teks dan fokus pada kata-kata yang dianggap lebih informatif

In [ ]:
def removeStopwords(text):
    stop_words = set(stopwords.words('indonesian'))
    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)
df['Artikel'] = df['Artikel'].apply(removeStopwords)
df['Artikel']

0       panas bumi memiliki ketersediaan terbaik energ...
1       perkiraan pelonggaran basis poin desember piki...
2       kementerian pariwisata ekonomi kreatif kemenpa...
3       jalur hilir titik terdampak longsor ka kecepat...
4       permasalahan air bersih selayaknya tanggung be...
                              ...                        
1075    jakarta grup idola perempuan newjeans salah pe...
1076    jakarta film horor pemukiman setan rumah produ...
1077    jakarta aktor grey s anatomy patrick dempsey d...
1078                                                     
1079    pencinta komik jepang anime one piece mengeksp...
Name: Artikel, Length: 1080, dtype: object

- Stemming

> Sastrawi adalah sebuah proyek open-source yang menyediakan library untuk melakukan pemrosesan bahasa Indonesia, khususnya dalam konteks stemming atau pengembalian kata dasar

In [ ]:
!pip install Sastrawi

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
def stemming(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(text)
df['Artikel'] = df['Artikel'].apply(stemming)
df['Artikel']

In [ ]:
df['Artikel'].dropna(inplace=True)

In [ ]:
df['Label'].dropna(inplace=True)

## TF-IDF

TF-IDF (Term Frequency-Inverse Document Frequency) adalah metode yang digunakan mengevaluasi seberapa penting sebuah kata bagi sebuah dokumen dalam kumpulan dokumen dengan cara mengalikan berapa kali sebuah kata muncul dalam sebuah dokumen (Term Frequency) dengan frekuensi dokumen terbalik (Inverse Document Frequency) dari kata di seluruh set dokumen.




### Term Frequency (TF)

$\text{TF}(t, d) = \frac{\text{jumlah kemunculan term } t \text{ dalam dokumen } d}{\text{total jumlah term dalam dokumen } d}
$

### Inverse Document Frequency (IDF)

$\text{IDF}(t, D) = \log\left(\frac{\text{total jumlah dokumen dalam korpus } |D|}{\text{jumlah dokumen yang mengandung term } t + 1}\right)
$

### Term Frequency-Inverse Document Frequency (TF-IDF)

$\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)$


Penjelasan:

- t adalah term (kata kunci),
- d adalah dokumen,
- D adalah korpus (kumpulan dokumen),
- ∣D∣ adalah total jumlah dokumen dalam korpus.

In [ ]:
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [ ]:
finalResult = pd.DataFrame(df, columns=["Artikel"])
label = pd.DataFrame(df, columns=["Label"])
# finalResult

> Membuat matriks TF-IDF dari teks dalam kolom 'Join Words' dari dataframe finalResult. Pertama, dilakukan impor modul stopwords dari NLTK untuk mendapatkan daftar stopwords dalam bahasa Indonesia. Selanjutnya, dibuat tokenizer dengan RegexpTokenizer untuk memproses teks. Kemudian, menggunakan TfidfVectorizer dari scikit-learn, teks diubah menjadi representasi vektor TF-IDF. Proses ini mencakup pengonversian huruf menjadi huruf kecil, penggunaan stopwords, dan penerapan tokenizer. Matriks TF-IDF yang dihasilkan, disimpan dalam variabel tfidf_matrix

In [ ]:
stopwords = stopwords.words('indonesian')

tokenizer = RegexpTokenizer(r'\w+')
vectorizer = TfidfVectorizer(lowercase=True,
                        stop_words=stopwords,
                        tokenizer = tokenizer.tokenize)


tfidf_matrix = vectorizer.fit_transform(finalResult["Artikel"])
tfidf_terms = vectorizer.get_feature_names_out()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


### Vector Space Model (VSM)

> (Vector Space Model atau VSM) adalah representasi matematis dari dokumen-dokumen dalam suatu korpus atau koleksi teks. Dalam VSM, setiap dokumen direpresentasikan sebagai vektor dalam ruang vektor yang memiliki dimensi sebanding dengan jumlah kata unik dalam seluruh korpus. Setiap dimensi vektor mewakili kata tertentu, dan nilai dalam dimensi tersebut dapat mencerminkan berbagai informasi, seperti frekuensi kemunculan kata atau skor TF-IDF

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# Membuat objek CountVectorizer
coun_vect = CountVectorizer(stop_words=stopwords)
count_matrix = coun_vect.fit_transform(finalResult["Artikel"])
count_array = count_matrix.toarray()
# Membuat DataFrame dari array dengan kolom berdasarkan term dalam teks
df = pd.DataFrame(data=count_array,columns = coun_vect.vocabulary_.keys())
df

,panas,bumi,memiliki,ketersediaan,terbaik,energi,terbarukan,dikontroljakarta,direktur,utama,...,mann,piero,taruffi,luffy,bertualang,piracy,menjelajahi,bajak,hreeloita,shanti
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1076,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1077,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1078,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# menampilkan bobot dari masing-masing term
vsc = pd.DataFrame(data=tfidf_matrix.toarray(),columns = vectorizer.vocabulary_.keys())
print("\nTF-IDF Vectorizer\n")
vsc


TF-IDF Vectorizer



,panas,bumi,memiliki,ketersediaan,terbaik,energi,terbarukan,dikontroljakarta,direktur,utama,...,mann,piero,taruffi,luffy,bertualang,piracy,menjelajahi,bajak,hreeloita,shanti
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1076,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1077,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1078,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Klasifikasi Berita

> Klasifikasi berita adalah suatu bentuk tugas dalam bidang pemrosesan bahasa alami (Natural Language Processing) dan machine learning, di mana tujuan utamanya adalah untuk menentukan kategori atau label yang sesuai untuk suatu teks berdasarkan isinya

- Fungsi train_test_split digunakan untuk membagi dataset menjadi data latih dan data uji. Ini secara acak mengacak dan membagi data menjadi dua bagian, biasanya satu untuk melatih model machine learning dan yang lainnya untuk menguji kinerjanya.

- SVC adalah jenis algoritma pembelajaran terawasi yang digunakan untuk tugas klasifikasi. Ini bekerja dengan menemukan hiperplane yang terbaik memisahkan kelas-kelas berbeda di ruang fitur.

- MinMaxScaler adalah teknik pra-pemrosesan yang digunakan untuk menyesuaikan skala fitur numerik ke rentang tertentu, sering kali antara 0 dan 1. Ini adalah bagian dari modul pra-pemrosesan scikit-learn. Penyesuaian skala penting untuk algoritma yang sensitif terhadap skala fitur input, seperti mesin vektor pendukung (support vector machines/SVM).

- Fungsi classification_report menghasilkan laporan teks yang menunjukkan metrik klasifikasi utama, seperti presisi, recall, dan F1-score. Fungsi accuracy_score menghitung akurasi model.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score

### Split Dataset

> Pembagian dataset menjadi data latih (data train) dan data uji (data test) merupakan praktik umum dalam machine learning. Pembagian ini dilakukan untuk melatih model pada sebagian data dan menguji kinerjanya pada sebagian data lainnya. Pada kasus ini, pembagian dilakukan dengan rasio 80:20, yang berarti 80% dari dataset digunakan untuk melatih model dan 20% digunakan untuk menguji model.

In [ ]:
# Label
X = tfidf_matrix
y = label

# Bagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Reduksi Dimensi (PCA)

> Reduksi dimenasi adalah teknik yang umum digunakan dalam analisis data dan machine learning untuk mengurangi jumlah fitur (dimensi) dalam dataset sambil mempertahankan sebanyak mungkin informasi

In [ ]:
from sklearn.decomposition import PCA

# Assuming X_train.toarray() has shape (number_of_samples, number_of_features)
valid_n_components = min(X_train.shape[0], X_train.shape[1])

# Use a value within the valid range for n_components
n_components = min(1000, valid_n_components)
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train.toarray())
X_test_pca = pca.transform(X_test.toarray())


scaler = MinMaxScaler()
X_train_pca = scaler.fit_transform(X_train_pca)
X_test_pca = scaler.transform(X_test_pca)

### Support Vector Machine (SVM)

Support Vector Machine (SVM) adalah algoritma machine learning yang digunakan untuk tugas klasifikasi dan regresi. Algoritma ini dapat melakukan pemisahan data yang kompleks dan nonlinier dengan mencari hyperplane optimal yang memisahkan dua kelas data (klasifikasi) atau mencari fungsi regresi yang sesuai dengan data.

Berikut rumus untuk melakukan klasifikasi menggunakan SVM sesuai kernel trick:
- Kernel Linear

> $K(x, z) = x^Tz$

- Kernel Radial Basis Function (RBF) atau Gaussian

> $K(x, z) = \exp\left(-\frac{||x - z||^2}{2\sigma^2}\right)$

- Polynomial

> $K(x, z) = (x^Tz + c)^d$

- Sigmoid

> $K(x, z) = \tanh(kx^Tz + c)$

In [ ]:
# Buat model SVM
svm_model = SVC(kernel='rbf', C=1, gamma=0.1)

# Latih model pada data latih
svm_model.fit(X_train_pca, y_train.values.ravel())

# Lakukan prediksi pada data uji
y_pred_svm = svm_model.predict(X_test_pca)

# Evaluasi performa model SVM
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print("\nSVM Classification Report:\n", classification_report(y_test, y_pred_svm, zero_division=1))


SVM Accuracy: 0.8981481481481481

SVM Classification Report:
               precision    recall  f1-score   support

     ekonomi       1.00      0.69      0.82        71
     hiburan       0.94      1.00      0.97        77
    olahraga       0.80      1.00      0.89        68

    accuracy                           0.90       216
   macro avg       0.91      0.90      0.89       216
weighted avg       0.92      0.90      0.89       216



### Naive Bayes

Algoritma Naive Bayes adalah metode klasifikasi yang berdasarkan pada teorema Bayes dengan asumsi independensi antar prediktor. Dalam kata lain, model Naive Bayes mengasumsikan bahwa keberadaan suatu fitur dalam kelas tertentu tidak berhubungan dengan keberadaan fitur lainnya. Meskipun asumsi ini jarang terpenuhi dalam dunia nyata, algoritma Naive Bayes telah terbukti cukup efektif dalam berbagai aplikasi, termasuk klasifikasi teks dan deteksi spam.

Algoritma ini memulai dengan menghitung probabilitas prior dari setiap kelas, yang merupakan frekuensi relatif kelas dalam dataset pelatihan. Kemudian, untuk setiap fitur dalam data, algoritma menghitung probabilitas kondisional fitur tersebut, mengingat setiap kelas. Probabilitas ini kemudian digunakan untuk memperbarui probabilitas prior menjadi probabilitas posterior menggunakan teorema Bayes. Akhirnya, algoritma memilih kelas dengan probabilitas posterior tertinggi sebagai prediksi untuk instance baru.

Berikut rumus untuk melakukan klasifikasi menggunakan Naive Bayes:

$P(y|X) = \frac{P(X|y) \cdot P(y)}{P(X)}$

di mana:

$P(y∣X)$ adalah probabilitas posterior dari kelas (target) dengan kondisi fitur.


$P(y)$ adalah probabilitas prior dari kelas.


$P(X∣y)$ adalah likelihood yang merupakan probabilitas fitur dengan kondisi kelas.


$P(X)$ adalah probabilitas prior dari fitur.

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Buat model Naive Bayes
naive_bayes_model = MultinomialNB()

# Latih model pada data latih
naive_bayes_model.fit(X_train_pca, y_train.values.ravel())

# Lakukan prediksi pada data uji
y_pred_nb = naive_bayes_model.predict(X_test_pca)

# Evaluasi performa model Naive Bayes
print("\nNaive Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))
print("\nNaive Bayes Classification Report:\n", classification_report(y_test, y_pred_nb, zero_division=1))


Naive Bayes Accuracy: 0.9861111111111112

Naive Bayes Classification Report:
               precision    recall  f1-score   support

     ekonomi       0.96      1.00      0.98        71
     hiburan       1.00      0.97      0.99        77
    olahraga       1.00      0.99      0.99        68

    accuracy                           0.99       216
   macro avg       0.99      0.99      0.99       216
weighted avg       0.99      0.99      0.99       216



### K-Nearest Neighbor (KNN)

Algoritma k-Nearest Neighbors (k-NN) adalah metode klasifikasi dan regresi yang sederhana dan intuitif. Dalam konteks klasifikasi, algoritma ini bekerja dengan mengidentifikasi `k` sampel pelatihan yang paling dekat dengan titik yang tidak diketahui dan menetapkan label dari mayoritas sampel terdekat tersebut ke titik yang tidak diketahui.

Berikut adalah rumus jarak Euclidean yang sering digunakan dalam algoritma k-NN untuk menghitung jarak antara dua titik, `x` dan `y`, dalam ruang berdimensi `d`:

$$ d(x, y) = \sqrt{\sum_{i=1}^{d} (x_i - y_i)^2} $$

Dalam rumus ini, `x_i` dan `y_i` adalah koordinat titik `x` dan `y` pada dimensi `i`.

Setelah jarak ke semua titik pelatihan dihitung, `k` titik dengan jarak terkecil dipilih. Dalam kasus klasifikasi, label kelas titik yang tidak diketahui ditentukan oleh voting mayoritas dari `k` titik terdekat. Dalam kasus regresi, nilai rata-rata atau median dari `k` titik terdekat digunakan sebagai prediksi.

Algoritma k-NN adalah metode berbasis instans yang tidak membuat asumsi apapun tentang distribusi data, menjadikannya pilihan yang baik untuk masalah di mana batas keputusan tidak mudah dipahami. Namun, algoritma ini dapat menjadi sangat lambat ketika jumlah sampel pelatihan besar, dan ia sensitif terhadap pilihan `k` dan metrik jarak yang digunakan.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Buat model KNN
knn_model = KNeighborsClassifier(n_neighbors=134)

# Latih model pada data latih
knn_model.fit(X_train_pca, y_train.values.ravel())

# Lakukan prediksi pada data uji
y_pred_knn = knn_model.predict(X_test_pca)

# Evaluasi performa model KNN
print("\nKNN Accuracy:", accuracy_score(y_test, y_pred_knn))
print("\nKNN Classification Report:\n", classification_report(y_test, y_pred_knn, zero_division=1))


KNN Accuracy: 0.5462962962962963

KNN Classification Report:
               precision    recall  f1-score   support

     ekonomi       1.00      0.00      0.00        71
     hiburan       0.75      0.68      0.71        77
    olahraga       0.45      0.97      0.61        68

    accuracy                           0.55       216
   macro avg       0.73      0.55      0.44       216
weighted avg       0.74      0.55      0.45       216



### Testing

Bagian ini akan memprediksi suatu inputan berita termasuk ke kategori ekonomi, hiburan, atau olahraga

In [ ]:
X_new = ['Surabaya, Jatim (ANTARA) - Dinas Koperasi Usaha Kecil dan Menengah dan Perdagangan (Dinkopumdag) Kota Surabaya, Jawa Timur, menjaga stabilisasi harga bahan pangan menjelang Natal 2023 dan Tahun Baru 2024 dengan meluncurkan aplikasi Cek Harga Kulak Bahan Pokok atau Cak Bapok. Kepala Dinkopumdag Kota Surabaya Dewi Soeryowati dalam keterangannya di Surabaya, Jatim, Sabtu, mengatakan pedagang pasar dan toko kelontong merupakan kategori yang mendapatkan prioritas pada aplikasi Cak Bapok, lantaran acap kali terdampak kenaikan harga bahan pangan. "Nanti, muncul semua komoditas kebutuhan bahan pokok, sehingga pedagang mendapatkan pilihan informasi barang dan harga yang paling kompetitif," katanya. Saat ini, lanjut dia, sudah terdapat empat distributor besar yang bergabung di aplikasi Cak Bapok yakni Rumah Potong Hewan (RPH), Pasar Induk Sidotopo Surabaya (PISS), Badan Urusan Logistik (Bulog), dan Koperasi Bina Raharja. "Ini untuk memutus mata rantai yang terlalu banyak dari tangan ke tangan dan memastikan harganya lebih murah," ucapnya. Ke depannya, Dinkopumdag menggandeng distributor lainnya, salah satunya petani sayuran binaan Dinas Ketahanan Pangan dan Pertanian (DKPP) setempat, agar lebih banyak mewadahi kebutuhan pasar. Dewi menjelaskan aplikasi itu bisa dimanfaatkan pedagang dan pelaku usaha mikro, kecil, dan menengah (UMKM) untuk kulakan barang dagangan. Karena itu, Cak Bapok dirancang dengan konsep satu kali klik untuk memudahkan dalam proses pemesanan barang. Bahkan, di dalam aplikasi ini juga ada tools Indeks Kecukupan Pangan (IKP) di Kota Surabaya. "Kalai berbicara komoditas beras di aplikasi ini bisa tahu kebutuhan dan ketersediaannya hingga saat ini berapa, sehingga kami bisa bilang ketersediaan pangan beras di Surabaya aman, karena sudah punya data riil di dalam aplikasi itu," ujarnya. Sedangkan, pengiriman barangnya menggunakan jasa kurir yang berstatus sebagai warga miskin dan telah tercatat di data base milik Pemkot Surabaya. "Kami berharap dengan menjadi kurir di Cak Bapok mereka dapat menambah penghasilan lebih dan bahkan, kalau bisa lepas dari garis kemiskinan. Ini sejalan dengan program pemkot dalam rangka mengentas kemiskinan di Surabaya," tutur Dewi.Pewarta: Abdul Hakim/Ananto PradanaEditor: Kelik Dewanto COPYRIGHT © ANTARA 2023']
X_new_tfidf = vectorizer.transform(X_new)

X_new_pca = pca.transform(X_new_tfidf.toarray())

pedict = naive_bayes_model.predict(X_new_pca)
print(pedict)

['ekonomi']
